In [ ]:
import requests
import hashlib
from datetime import datetime, timedelta
from pyspark.sql import functions as F
from pyspark.sql.types import *

## Parameters

In [ ]:
# Parameters (can be passed from Pipeline)
fabric_gps_api_url = "https://fabric-gps.com/api/releases"
modified_within_days = 90  # Get features modified in last N days (1-30 per API, but we'll fetch all)
page_size = 200  # Max allowed by API
include_planned = True  # Include planned features (not yet shipped)
include_shipped = True  # Include shipped features

## Step 1: Fetch Feature Releases from Fabric GPS API

In [ ]:
def fetch_all_fabric_gps_releases(base_url, page_size=200, modified_within_days=None, include_planned=True, include_shipped=True):
    """
    Fetch all releases from Fabric GPS API with pagination
    Returns list of release objects
    """
    all_releases = []
    page = 1
    
    print(f"🔄 Fetching from Fabric GPS API: {base_url}")
    
    while True:
        try:
            # Build query parameters
            params = {
                "page": page,
                "page_size": page_size
            }
            
            if modified_within_days and modified_within_days <= 30:
                params["modified_within_days"] = modified_within_days
            
            # Make request
            response = requests.get(base_url, params=params, timeout=30)
            response.raise_for_status()
            data = response.json()
            
            # Extract releases
            releases = data.get("data", [])
            
            if not releases:
                print(f"  → No more data on page {page}")
                break
            
            # Filter by release_status if specified
            filtered_releases = []
            for release in releases:
                status = release.get("release_status", "")
                
                # Apply filters
                if status == "Planned" and not include_planned:
                    continue
                if status == "Shipped" and not include_shipped:
                    continue
                    
                filtered_releases.append(release)
            
            all_releases.extend(filtered_releases)
            
            # Check pagination
            pagination = data.get("pagination", {})
            has_next = pagination.get("has_next", False)
            total_items = pagination.get("total_items", 0)
            
            print(f"  → Page {page}: Fetched {len(filtered_releases)} releases (Total: {len(all_releases)}/{total_items})")
            
            if not has_next:
                print(f"  ✅ Reached end of data")
                break
            
            page += 1
            
        except Exception as e:
            print(f"  ❌ Error fetching page {page}: {e}")
            break
    
    print(f"\n✅ Total releases fetched: {len(all_releases)}")
    return all_releases

# Fetch all releases
releases_data = fetch_all_fabric_gps_releases(
    fabric_gps_api_url, 
    page_size=page_size,
    modified_within_days=modified_within_days if modified_within_days <= 30 else None,
    include_planned=include_planned,
    include_shipped=include_shipped
)

print(f"\n📋 Sample releases:")
for release in releases_data[:3]:
    print(f"  - {release.get('feature_name')} ({release.get('release_type')}) - {release.get('product_name')}")

## Step 2: Transform to FUAM Schema

In [ ]:
def transform_fabric_gps_to_fuam(releases):
    """
    Transform Fabric GPS release data to FUAM feature_releases schema
    """
    transformed = []
    
    for release in releases:
        # Parse release date
        release_date_str = release.get("release_date")
        try:
            if release_date_str:
                release_date = datetime.strptime(release_date_str, "%Y-%m-%d")
            else:
                release_date = None  # Planned features may not have a date yet
        except:
            release_date = None
        
        # Parse last modified
        last_modified_str = release.get("last_modified")
        try:
            last_modified = datetime.strptime(last_modified_str, "%Y-%m-%d") if last_modified_str else datetime.now()
        except:
            last_modified = datetime.now()
        
        # Determine if preview
        release_type = release.get("release_type", "")
        is_preview = "preview" in release_type.lower()
        
        # Map product_name to workload (align with FUAM naming)
        product_name = release.get("product_name", "Unknown")
        workload_mapping = {
            "Power BI": "Power BI",
            "Data Factory": "Data Factory",
            "Data Engineering": "Data Engineering",
            "Data Science": "Data Science",
            "Data Warehouse": "Data Warehouse",
            "Real-Time Intelligence": "Real-Time Intelligence",
            "OneLake": "Data Engineering",
            "Administration, Governance and Security": "Governance",
            "Cosmos DB (NoSQL)": "Cosmos DB"
        }
        workload = workload_mapping.get(product_name, product_name)
        
        # Create feature record
        feature = {
            "feature_id": release.get("release_item_id"),  # Use Fabric GPS UUID
            "feature_name": release.get("feature_name", "Unknown"),
            "feature_description": release.get("feature_description"),
            "workload": workload,
            "product_name": product_name,
            "release_date": release_date,
            "release_type": release_type,
            "release_status": release.get("release_status", "Unknown"),
            "is_preview": is_preview,
            "is_planned": release.get("release_status") == "Planned",
            "is_shipped": release.get("release_status") == "Shipped",
            "last_modified": last_modified,
            "source_url": f"https://fabric-gps.com/api/releases?release_item_id={release.get('release_item_id')}",
            "source": "Fabric GPS API",
            "extracted_date": datetime.now()
        }
        
        transformed.append(feature)
    
    return transformed

print("🔄 Transforming releases to FUAM schema...")
features_data = transform_fabric_gps_to_fuam(releases_data)
print(f"✅ Transformed {len(features_data)} features")

# Statistics
preview_count = sum(1 for f in features_data if f["is_preview"])
planned_count = sum(1 for f in features_data if f["is_planned"])
shipped_count = sum(1 for f in features_data if f["is_shipped"])

print(f"\n📊 Feature Breakdown:")
print(f"  - Preview features: {preview_count}")
print(f"  - Planned features: {planned_count}")
print(f"  - Shipped features: {shipped_count}")

## Step 3: Create DataFrame

In [ ]:
schema = StructType([
    StructField("feature_id", StringType(), False),
    StructField("feature_name", StringType(), False),
    StructField("feature_description", StringType(), True),
    StructField("workload", StringType(), True),
    StructField("product_name", StringType(), True),
    StructField("release_date", TimestampType(), True),  # Nullable for planned features
    StructField("release_type", StringType(), True),
    StructField("release_status", StringType(), True),
    StructField("is_preview", BooleanType(), False),
    StructField("is_planned", BooleanType(), False),
    StructField("is_shipped", BooleanType(), False),
    StructField("last_modified", TimestampType(), False),
    StructField("source_url", StringType(), True),
    StructField("source", StringType(), True),
    StructField("extracted_date", TimestampType(), False)
])

df_features = spark.createDataFrame(features_data, schema=schema)

print(f"✅ Created DataFrame with {df_features.count()} rows")
df_features.show(5, truncate=False)

## Step 4: Write to Delta Lake

In [ ]:
# Define table path
table_name = "feature_releases_roadmap"
table_path = f"Tables/{table_name}"

print(f"🔄 Writing to Delta table: {table_name}")

# Write with merge logic (upsert)
try:
    from delta.tables import DeltaTable
    
    # Check if table exists
    if DeltaTable.isDeltaTable(spark, table_path):
        print("  → Table exists, performing MERGE (upsert)...")
        
        delta_table = DeltaTable.forPath(spark, table_path)
        
        # Merge: Update existing, insert new
        # Use last_modified to track changes
        delta_table.alias("target").merge(
            df_features.alias("source"),
            "target.feature_id = source.feature_id"
        ).whenMatchedUpdate(
            condition="source.last_modified > target.last_modified",
            set={
                "feature_name": "source.feature_name",
                "feature_description": "source.feature_description",
                "workload": "source.workload",
                "product_name": "source.product_name",
                "release_date": "source.release_date",
                "release_type": "source.release_type",
                "release_status": "source.release_status",
                "is_preview": "source.is_preview",
                "is_planned": "source.is_planned",
                "is_shipped": "source.is_shipped",
                "last_modified": "source.last_modified",
                "source_url": "source.source_url",
                "extracted_date": "source.extracted_date"
            }
        ).whenNotMatchedInsertAll(
        ).execute()
        
        print("  ✅ MERGE completed")
    else:
        print("  → Table doesn't exist, creating new table...")
        df_features.write.format("delta").mode("overwrite").save(table_path)
        print("  ✅ Table created")
    
    # Show final count
    final_count = spark.read.format("delta").load(table_path).count()
    print(f"\n✅ Total records in {table_name}: {final_count}")
    
except Exception as e:
    print(f"❌ Error writing to Delta: {e}")
    raise

## Step 5: Summary Statistics

In [ ]:
print("\n📊 Feature Release Roadmap Statistics:")
print("=" * 70)

df_summary = spark.read.format("delta").load(table_path)

# By workload/product
print("\n🔸 By Workload:")
df_summary.groupBy("workload").count().orderBy(F.desc("count")).show(truncate=False)

# By release type
print("\n🔸 By Release Type:")
df_summary.groupBy("release_type").count().orderBy(F.desc("count")).show(truncate=False)

# By release status
print("\n🔸 By Release Status:")
df_summary.groupBy("release_status").count().orderBy(F.desc("count")).show(truncate=False)

# Preview vs GA
print("\n🔸 Preview vs GA:")
df_summary.groupBy("is_preview").count().show(truncate=False)

# Planned vs Shipped
print("\n🔸 Planned vs Shipped:")
df_summary.select("is_planned", "is_shipped").groupBy("is_planned", "is_shipped").count().show(truncate=False)

# Recent modifications (last 30 days)
recent_cutoff = datetime.now() - timedelta(days=30)
recent_count = df_summary.filter(F.col("last_modified") >= recent_cutoff).count()
print(f"\n🔸 Features modified in last 30 days: {recent_count}")

# Upcoming releases (planned)
planned_count = df_summary.filter(F.col("is_planned") == True).count()
print(f"🔸 Planned features (roadmap): {planned_count}")

# Show upcoming preview features
print("\n🔸 Upcoming Preview Features (sample):")
df_summary.filter((F.col("is_preview") == True) & (F.col("is_planned") == True)) \
    .select("feature_name", "product_name", "release_date", "release_status") \
    .orderBy(F.col("release_date").asc()) \
    .show(10, truncate=60)

print("\n" + "=" * 70)
print("✅ Transfer Feature Releases GpsApi Unit - COMPLETED")
print(f"💡 Tip: Use this table for roadmap planning and preview feature tracking")